In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
import time

def scrape_datacenters():
    # Set up the webdriver (ensure chromedriver is installed and its path is set correctly)
    service = Service('/Applications/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service)

    # Open the starting URL
    driver.get("https://datacenters.com/locations/united-states")
    wait = WebDriverWait(driver, 10)
    
    # This will store the extracted relative URLs
    extracted_urls = []

    while True:
        try:
            # Wait until the grid of <a> elements is present on the page.
            # The grid container has been determined by its full xpath:
            grid = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[1]")
            ))
            
            # Find all <a> elements within the grid container.
            a_tags = grid.find_elements(By.TAG_NAME, "a")
            
            for a in a_tags:
                href = a.get_attribute("href")
                if href:
                    # Parse the URL to extract the path (i.e. the relative URL like "/microsoft-azure-east-us-virginia")
                    path = urlparse(href).path
                    if path not in extracted_urls:
                        extracted_urls.append(path)
                        print(path)

            # Now, locate the "next page" toggle button by its xpath.
            next_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[5]")
            ))
            
            # Check if the next button is disabled.
            # (Sometimes disabled buttons have a "disabled" attribute or a specific class you can check.)
            if next_button.get_attribute("disabled"):
                print("Reached the last page.")
                break

            # Click the next page button.
            next_button.click()

            # Pause briefly to allow the next page to load.
            time.sleep(2)
            
        except Exception as e:
            # If an exception is thrown (for example, if the next button is not found), exit the loop.
            print("Exception encountered:", e)
            break

    driver.quit()
    return extracted_urls


In [7]:
links = scrape_datacenters()

/microsoft-azure-east-us-virginia
/equinix-dc2-ashburn
/google-mayes-county-ok
/equinix-ny4-secaucus
/microsoft-azure-des-moines
/microsoft-azure-west-us-2-washington
/microsoft-azure-east-us-2-virginia
/qts-richmond
/amazon-aws-pdx4-boardman-or
/google-midlothian-tx
/microsoft-azure-south-central-us-texas
/equinix-mi1-miami
/microsoft-azure-north-central-us-illinois
/apple-inc-maiden
/switch-switch-core-campus
/switch-switch-citadel-campus
/google-montgomery-county-tn
/google-the-dalles-or
/switch-switch-keep
/facebook-newton-atlanta
/oracle-us-east-us-ashburn-1
/equinix-ny5-secaucus
/equinix-ch3-elk-grove
/qts-phoenix
/apple-inc-waukee-ia
/facebook-huntsville
/amazon-aws-iad23-ashburn
/apple-inc-mesa
/facebook-los-lunas
/google-papillion-ne
/equinix-ny2-secaucus
/google-douglas-county-ga
/amazon-aws-port-of-morrow-3
/google-henderson-nv
/amazon-aws-13600-eds-drive
/apple-inc-prineville
/google-new-albany-oh
/google-lenoir-nc
/cyrusone-nym1-somerset
/oracle-us-west-us-phoenix-1
Except

In [28]:
def scrape_datacenters():
    # Update this path to where your chromedriver is located
    service = Service('/Applications/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service)
    
    driver.get("https://datacenters.com/locations/united-states")
    wait = WebDriverWait(driver, 10)
    extracted_urls = []
    
    # OPTIONAL: If a cookie consent banner appears, try to dismiss it.
    try:
        cookie_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(., 'Accept')]")
        ))
        cookie_button.click()
        print("Cookie banner accepted.")
    except Exception as e:
        print("Cookie banner not found or already handled:", e)
    
    # We'll count the pages as we loop
    page_count = 1
    
    while True:
        try:
            print(f"\nProcessing page {page_count}...")
            # Wait until the grid containing the data center links is present.
            grid = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[1]")
            ))
            a_tags = grid.find_elements(By.TAG_NAME, "a")
            
            for a in a_tags:
                href = a.get_attribute("href")
                if href:
                    # Extract just the path part from the URL.
                    path = urlparse(href).path
                    if path not in extracted_urls:
                        extracted_urls.append(path)
                        print(path)
            
            # Determine the correct "next page" button XPath based on the current page.
            if page_count < 4:
                next_button_xpath = "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[5]"
            else:
                next_button_xpath = "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[9]"
            
            # Locate the "next page" button.
            next_button = wait.until(EC.presence_of_element_located(
                (By.XPATH, next_button_xpath)
            ))
            
            # --- Handle potential overlay interfering with the click ---
            try:
                overlay = driver.find_element(By.CLASS_NAME, "ot-sdk-row")
                # Remove the overlay using JavaScript.
                driver.execute_script("arguments[0].remove();", overlay)
                print("Overlay removed.")
            except Exception as e:
                print("No overlay found:", e)
            
            # Scroll the next button into view.
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Give time for the scroll
            
            # Click the next button using JavaScript to avoid interception.
            driver.execute_script("arguments[0].click();", next_button)
            
            # Wait for the next page to load.
            time.sleep(2)
            
            # Increment the page counter.
            page_count += 1
            
        except Exception as e:
            print("Exception encountered:", e)
            print("Assuming no more pages or a critical error occurred.")
            break

    driver.quit()
    return extracted_urls


In [29]:
urls = scrape_datacenters()

Cookie banner not found or already handled: Message: 
Stacktrace:
0   chromedriver                        0x000000010306f94c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x0000000103067fa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x0000000102bcdff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000102c14d5c cxxbridge1$string$len + 383504
4   chromedriver                        0x0000000102c55f14 cxxbridge1$string$len + 650184
5   chromedriver                        0x0000000102c09004 cxxbridge1$string$len + 335032
6   chromedriver                        0x00000001030381f4 cxxbridge1$str$ptr + 2501068
7   chromedriver                        0x000000010303b508 cxxbridge1$str$ptr + 2514144
8   chromedriver                        0x000000010301d8a4 cxxbridge1$str$ptr + 2392188
9   chromedriver                        0x000000010303bdb0 cxxbridge1$str$ptr + 2516360
10  chromedriver                        0x00000

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
import time

def scrape_last_datacenters():
    # Update this path to your chromedriver executable
    service = Service('/Applications/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service)
    
    driver.get("https://datacenters.com/locations/united-states")
    wait = WebDriverWait(driver, 10)
    
    # We’ll store URLs from two sections:
    # 1. Pages 1-50 (extracted_urls_first)
    # 2. Pages 51-53 (extracted_urls_last)
    extracted_urls_first = []
    extracted_urls_last = []
    
    # --- Optional: Dismiss the cookie consent banner if it appears ---
    try:
        cookie_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(., 'Accept')]")
        ))
        cookie_button.click()
        print("Cookie banner accepted.")
    except Exception as e:
        print("Cookie banner not found or already handled:", e)
    
    # ======================================================
    # Part 1: Process Pages 1-50
    # ======================================================
    page_count = 1
    while page_count <= 50:
        print(f"\nProcessing page {page_count} (pages 1-50)...")
        try:
            # If this is not the last page in the first block, click the next button.
            if page_count < 50:
                # Use one XPath for pages 1-3, and a different one for pages 4-50.
                if page_count < 4:
                    next_button_xpath = "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[5]"
                else:
                    next_button_xpath = "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[9]"
                
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
                
                # Remove any overlay that might block the button.
                try:
                    overlay = driver.find_element(By.CLASS_NAME, "ot-sdk-row")
                    driver.execute_script("arguments[0].remove();", overlay)
                    print("Overlay removed.")
                except Exception as e:
                    print("No overlay found:", e)
                
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(1)  # Give time for scrolling
                driver.execute_script("arguments[0].click();", next_button)
                print(f"Clicked next page for page {page_count}.")
                time.sleep(2)  # Wait for the next page to load
        except Exception as e:
            print("Exception encountered on page", page_count, ":", e)
            print("Stopping pagination for pages 1-50.")
            break
        
        page_count += 1

    # ======================================================
    # Part 2: Process Pages 51-53
    # ------------------------------------------------------
    # Now the driver is on page 50. We want to get pages 51-53,
    # so for each of these pages, first click the next button 
    # using the new XPath, then extract the new links.
    # Note: We do NOT extract links from page 50 again.
    # ======================================================
    for page in range(51, 54):
        print(f"\nProcessing page {page} (pages 51-53)...")
        try:
            # Click the next button using the new configuration for the last pages.
            # This XPath applies for pages 51 and 52 (to move to the next page).
            next_button_xpath = "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[2]/div[1]/nav/button[4]"
            next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
            
            try:
                overlay = driver.find_element(By.CLASS_NAME, "ot-sdk-row")
                driver.execute_script("arguments[0].remove();", overlay)
                print("Overlay removed.")
            except Exception as e:
                print("No overlay found:", e)
            
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_button)
            print(f"Clicked next page to move to page {page}.")
            time.sleep(2)  # Wait for the new page to load

            # Now extract links from the new page (pages 51-53).
            grid = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[2]/div[3]/div[1]")
            ))
            a_tags = grid.find_elements(By.TAG_NAME, "a")
            for a in a_tags:
                href = a.get_attribute("href")
                if href:
                    path = urlparse(href).path
                    if path not in extracted_urls_last:
                        extracted_urls_last.append(path)
                        print(path)
        except Exception as e:
            print("Exception encountered on page", page, ":", e)
            print("Stopping pagination for pages 51-53.")
            break

    driver.quit()
    return extracted_urls_first, extracted_urls_last


In [40]:
last_urls = scrape_last_datacenters()

Cookie banner not found or already handled: Message: 
Stacktrace:
0   chromedriver                        0x0000000102aeb94c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x0000000102ae3fa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x0000000102649ff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000102690d5c cxxbridge1$string$len + 383504
4   chromedriver                        0x00000001026d1f14 cxxbridge1$string$len + 650184
5   chromedriver                        0x0000000102685004 cxxbridge1$string$len + 335032
6   chromedriver                        0x0000000102ab41f4 cxxbridge1$str$ptr + 2501068
7   chromedriver                        0x0000000102ab7508 cxxbridge1$str$ptr + 2514144
8   chromedriver                        0x0000000102a998a4 cxxbridge1$str$ptr + 2392188
9   chromedriver                        0x0000000102ab7db0 cxxbridge1$str$ptr + 2516360
10  chromedriver                        0x00000

In [55]:
all_urls = urls + last_urls

# Scraping the DC Profiles

In [63]:
def scrape_datacenter_details(url_extensions):
    # Initialize the Chrome driver.
    service = Service('/Applications/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 10)
    
    base_url = "https://datacenters.com"
    results = []
    
    for extension in url_extensions:
        full_url = base_url + extension
        print(f"Processing: {full_url}")
        driver.get(full_url)
        time.sleep(1)  # Let the page start loading
        
        # Get the data center name from the <h1> element.
        try:
            name_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[3]/div/h1")
            ))
            name = name_element.text.strip()
        except Exception as e:
            print(f"Error getting name from {full_url}: {e}")
            name = None

        # Get the address from the designated <span>.
        try:
            address_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/div[1]/span[2]")
            ))
            address = address_element.text.strip()
        except Exception as e:
            print(f"Error getting address from {full_url}: {e}")
            address = None

        # Get the sidebar data fields. They are contained within a specific div.
        try:
            sidebar = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[2]/div/div[2]")
            ))
            # Get all immediate child div elements (each one represents a data field)
            field_elements = sidebar.find_elements(By.XPATH, "./div")
            # Extract text from each element (ignoring empty strings)
            fields = [elem.text.strip() for elem in field_elements if elem.text.strip()]
        except Exception as e:
            print(f"Error getting sidebar fields from {full_url}: {e}")
            fields = []
        
        results.append({
            "url": full_url,
            "name": name,
            "address": address,
            "fields": fields
        })
        time.sleep(1)  # Optional pause between pages

    driver.quit()
    
    # Determine the maximum number of sidebar fields across all pages.
    max_fields = max(len(r["fields"]) for r in results) if results else 0
    
    # Build a list of dictionaries for the DataFrame.
    data_for_df = []
    for r in results:
        row = {
            "url": r["url"],
            "name": r["name"],
            "address": r["address"]
        }
        for i in range(max_fields):
            key = f"field{i+1}"
            row[key] = r["fields"][i] if i < len(r["fields"]) else None
        data_for_df.append(row)
    
    df = pd.DataFrame(data_for_df)
    return df

In [68]:
datacenters = scrape_datacenter_details(all_urls)

Processing: https://datacenters.com/microsoft-azure-east-us-virginia
Processing: https://datacenters.com/equinix-dc2-ashburn
Processing: https://datacenters.com/google-mayes-county-ok
Processing: https://datacenters.com/equinix-ny4-secaucus
Processing: https://datacenters.com/microsoft-azure-des-moines
Processing: https://datacenters.com/microsoft-azure-west-us-2-washington
Processing: https://datacenters.com/microsoft-azure-east-us-2-virginia
Processing: https://datacenters.com/qts-richmond
Processing: https://datacenters.com/amazon-aws-pdx4-boardman-or
Processing: https://datacenters.com/google-midlothian-tx
Processing: https://datacenters.com/microsoft-azure-south-central-us-texas
Processing: https://datacenters.com/equinix-mi1-miami
Processing: https://datacenters.com/microsoft-azure-north-central-us-illinois
Processing: https://datacenters.com/apple-inc-maiden
Processing: https://datacenters.com/switch-switch-core-campus
Error getting name from https://datacenters.com/switch-switc

KeyboardInterrupt: 

In [69]:
def scrape_datacenter_details(url_extensions):
    # Initialize the Chrome driver.
    service = Service('/Applications/chromedriver-mac-arm64/chromedriver')
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 10)
    
    base_url = "https://datacenters.com"
    results = []
    
    for extension in url_extensions:
        full_url = base_url + extension
        print(f"Processing: {full_url}")
        driver.get(full_url)
        time.sleep(1)  # Allow the page to start loading
        
        # --- Extract the Data Center Name ---
        # Try multiple XPaths to account for layout variations.
        name = None
        heading_xpaths = [
            "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[3]/div/h1",  # Standard case
            "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/h1"                # Fallback case
        ]
        for xpath in heading_xpaths:
            try:
                name_element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                name = name_element.text.strip()
                if name:
                    break  # Use the first non-empty heading found.
            except Exception as e:
                print(f"Heading not found using {xpath}.")
        if not name:
            print("Error extracting heading from:", full_url)
        
        # --- Extract the Address ---
        try:
            address_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]/div[1]/span[2]")
            ))
            address = address_element.text.strip()
        except Exception as e:
            print(f"Error getting address from {full_url}: {e}")
            address = None

        # --- Extract the Sidebar Data Fields ---
        try:
            sidebar = wait.until(EC.presence_of_element_located(
                (By.XPATH, "/html/body/div[1]/div[4]/div/div[1]/div[1]/div/div[2]/div/div[2]")
            ))
            # Get all immediate child <div> elements
            field_elements = sidebar.find_elements(By.XPATH, "./div")
            fields = [elem.text.strip() for elem in field_elements if elem.text.strip()]
        except Exception as e:
            print(f"Error getting sidebar fields from {full_url}: {e}")
            fields = []
        
        results.append({
            "url": full_url,
            "name": name,
            "address": address,
            "fields": fields
        })
        time.sleep(1)  # Pause between pages

    driver.quit()
    
    # Determine the maximum number of sidebar fields across all pages.
    max_fields = max(len(r["fields"]) for r in results) if results else 0
    
    # Build a list of dictionaries for the DataFrame.
    data_for_df = []
    for r in results:
        row = {
            "url": r["url"],
            "name": r["name"],
            "address": r["address"]
        }
        for i in range(max_fields):
            key = f"field{i+1}"
            row[key] = r["fields"][i] if i < len(r["fields"]) else None
        data_for_df.append(row)
    
    df = pd.DataFrame(data_for_df)
    return df


In [75]:
datacenters = scrape_datacenter_details(all_urls)

Processing: https://datacenters.com/microsoft-azure-east-us-virginia
Processing: https://datacenters.com/equinix-dc2-ashburn
Processing: https://datacenters.com/google-mayes-county-ok
Processing: https://datacenters.com/equinix-ny4-secaucus
Processing: https://datacenters.com/microsoft-azure-des-moines
Processing: https://datacenters.com/microsoft-azure-west-us-2-washington
Processing: https://datacenters.com/microsoft-azure-east-us-2-virginia
Processing: https://datacenters.com/qts-richmond
Processing: https://datacenters.com/amazon-aws-pdx4-boardman-or
Processing: https://datacenters.com/google-midlothian-tx
Processing: https://datacenters.com/microsoft-azure-south-central-us-texas
Processing: https://datacenters.com/equinix-mi1-miami
Processing: https://datacenters.com/microsoft-azure-north-central-us-illinois
Processing: https://datacenters.com/apple-inc-maiden
Processing: https://datacenters.com/switch-switch-core-campus
Heading not found using /html/body/div[1]/div[4]/div/div[1]/

In [76]:
datacenters

,url,name,address,field1,field2,field3,field4,field5
0,https://datacenters.com/microsoft-azure-east-u...,Microsoft Azure: East US-Virginia,"101 Herbert Drive, Boydton, VA, USA",1100000 sqft total space,0 colocation products,175.23 miles to nearest airport,None,None
1,https://datacenters.com/equinix-dc2-ashburn,Equinix: DC2 Ashburn Data Center,"21715 Filigree Court, Ashburn, VA, USA",147600 sqft total space,118447 sqft colocation space,4.9 MW total power,0 colocation products,4.35 miles to nearest airport
2,https://datacenters.com/google-mayes-county-ok,"Google: Mayes County, OK Data Center",4111 North East 1st Street,1400000 sqft total space,0 colocation products,34.8 miles to nearest airport,None,None
3,https://datacenters.com/equinix-ny4-secaucus,Equinix: NY4 Secaucus Data Center,"755 Secaucus Road, Secaucus, NJ, USA",338967 sqft total space,151772 sqft colocation space,18.5 MW total power,0 colocation products,7.46 miles to nearest airport
4,https://datacenters.com/microsoft-azure-des-mo...,Microsoft Azure: Des Moines Data Center,"550 White Crane Rd, West Des Moines, IA 50265,...",408000 sqft total space,0 colocation products,2.49 miles to nearest airport,None,None
...,...,...,...,...,...,...,...,...
2054,https://datacenters.com/amazon-aws-crosby-campus,Amazon AWS: Crosby Campus,"6645 Crosby Court, Dublin, OH, USA",0 colocation products,2.76 miles to nearest airport,None,None,None
2055,https://datacenters.com/tecfusions-new-kensing...,"TECfusions: New Kensington, PA Data Center","100 Technical Drive, New Kensington, PA, USA","500,000 sqft total space","500,000 sqft colocation space",200 MW total power,0 colocation products,5.51 miles to nearest airport
2056,https://datacenters.com/navisite-santa-clara,NaviSite: Santa Clara Data Center,"2720 Zanker Road, San Jose, CA, USA",1.2 MW total power,0 colocation products,1.9 miles to nearest airport,None,None
2057,https://datacenters.com/opti9-technologies-neb...,Opti9 Technologies: Nebraska Data Center,"1010 N 96th St ste 100, Omaha, NE 68114, USA",0 colocation products,2.71 miles to nearest airport,None,None,None
